# AIM: 이미지 평가
- 주제(topic), 프롬프트(prompt), 이미지들을 업로드하고 기준 이미지 선택
- Hugging Face 토큰은 `.env`의 `HUGGING_FACE_API` 또는 `HF_TOKEN` 사용


In [11]:
# 환경 준비: 프로젝트 루트 경로 추가 및 .env 로딩
import os, sys
from dotenv import load_dotenv, find_dotenv
ROOT = os.path.dirname(os.path.dirname(os.getcwd())) if os.path.basename(os.getcwd())=='notebooks' else os.getcwd()
if ROOT not in sys.path: sys.path.insert(0, ROOT)
load_dotenv(find_dotenv())
print('ROOT =', ROOT)


ROOT = /home/jovyan/Desktop/aim


In [ ]:
# 위젯 UI 및 썸네일 프리뷰
import ipywidgets as widgets
from IPython.display import display, clear_output
import io
from PIL import Image as PILImage

# 입력창 크기 확대
topic_w = widgets.Textarea(
    value='A wooden cabin with a smoking chimney stands among snow-covered trees and a snowy mountain',
    description='Topic',
    style={'description_width':'initial'},
    layout=widgets.Layout(width='800px', height='80px')
)

prompt_w = widgets.Textarea(
    value='Only one wooden cabin with one chimney, no other buildings or cabins, in a snowy forest with mountains',
    description='Prompt',
    style={'description_width':'initial'},
    layout=widgets.Layout(width='800px', height='80px')
)

files_w = widgets.FileUpload(accept='image/*', multiple=True, description='Upload Images')
ref_dropdown = widgets.Dropdown(options=[], description='Reference Image', style={'description_width':'initial'})
enable_clip_w = widgets.Checkbox(value=True, description='Enable CLIP')
enable_blip2_w = widgets.Checkbox(value=True, description='Enable BLIP2')
enable_lpips_w = widgets.Checkbox(value=True, description='Enable LPIPS')
run_btn = widgets.Button(description='Run Evaluation', button_style='primary')
thumbs_out = widgets.Output()
out = widgets.Output()


In [17]:
# 평가 실행 로직
import tempfile, io
from PIL import Image
import pandas as pd
import evaluation

def run_eval_clicked(_):
    with out:
        clear_output()
        if not files_w.value:
            print('이미지를 업로드하세요.')
            return
        
        # 진행 상태 표시
        print("모델 로딩 중...")
        
        # 환경 플래그 설정
        os.environ['DISABLE_CLIP'] = '0' if enable_clip_w.value else '1'
        os.environ['DISABLE_BLIP2'] = '0' if enable_blip2_w.value else '1'
        os.environ['DISABLE_LPIPS'] = '0' if enable_lpips_w.value else '1'
        
        # 평가기 인스턴스
        eval_inst = evaluation.get_evaluator()
        
        print("이미지 처리 중...")
        
        # 업로드 파일을 임시경로에 저장
        tmpdir = tempfile.mkdtemp(prefix='aim_nb_')
        name_to_path = {}
        for i, item in enumerate(files_w.value):
            if hasattr(item, 'name') and item.name:
                name = item.name
            else:
                meta = item.get('metadata') or {}
                name = meta.get('name', f'image_{i+1}.png')
            
            content = item['content'] if isinstance(item['content'], (bytes, bytearray)) else item['content'].tobytes()
            img = Image.open(io.BytesIO(content)).convert('RGB')
            path = os.path.join(tmpdir, name)
            img.save(path)
            name_to_path[name] = path
            
        ref_name = ref_dropdown.value
        if ref_name not in name_to_path:
            print('기준 이미지가 목록에 없습니다.')
            return
            
        print("이미지 평가 중...")
        
        topic = topic_w.value
        prompt = prompt_w.value
        results = []
        
        # 기준 이미지는 LPIPS를 제외하고 평가
        for name, path in name_to_path.items():
            if name == ref_name:
                # 기준 이미지는 LPIPS 제외
                r = eval_inst.evaluate(path, name_to_path[ref_name], prompt, topic)
                r['lpips_score'] = None
                r['lpips_norm'] = None
                # 기준 이미지는 LPIPS 제외하고 final_score 재계산
                clip_norm = r['clip_norm']
                blip2_norm = r['blip2_norm']
                weights = {
                    'clip': 0.2 if enable_clip_w.value else 0.0,
                    'blip2': 0.5 if enable_blip2_w.value else 0.0,
                }
                weight_sum = sum(weights.values()) or 1.0
                score = (clip_norm * weights['clip'] + blip2_norm * weights['blip2'])
                r['final_score'] = (score / weight_sum) * 100
            else:
                # 다른 이미지는 전체 평가
                r = eval_inst.evaluate(path, name_to_path[ref_name], prompt, topic)
            
            r['filename'] = name
            results.append(r)
            
        if not results:
            print('평가 결과가 없습니다.')
            return
            
        print("평가 완료")
        
        # 결과 표시
        df = pd.DataFrame(results)[['filename','clip_score','clip_norm','blip2_similarity','blip2_norm','lpips_score','lpips_norm','final_score']]
        display(df)
        
       
run_btn.on_click(run_eval_clicked)
print('Ready.')


Ready.


In [18]:
def render_thumbnails(change=None):
    with thumbs_out:
        clear_output()
        if not files_w.value:
            print('이미지를 업로드하세요.')
            return
        items = []
        current_ref = ref_dropdown.value or ''
        for i, f in enumerate(files_w.value):
            # 파일명 추출 개선
            if hasattr(f, 'name') and f.name:
                name = f.name
            else:
                meta = f.get('metadata') or {}
                name = meta.get('name', f'image_{i+1}.png')
            
            content = f['content'] if isinstance(f['content'], (bytes, bytearray)) else f['content'].tobytes()
            img = PILImage.open(io.BytesIO(content)).convert('RGB')
            thumb = img.copy()
            thumb.thumbnail((160,160))
            buf = io.BytesIO()
            thumb.save(buf, format='PNG')
            wimg = widgets.Image(value=buf.getvalue(), format='png', width=160, height=160)
            border = '3px solid #0056b3' if name == current_ref else '1px solid #ccc'
            box = widgets.VBox([wimg, widgets.HTML(f"<div style='text-align:center; font-size:12px'>{name}</div>")],
                               layout=widgets.Layout(border=border, padding='4px', margin='4px', align_items='center', width='180px'))
            items.append(box)
        grid = widgets.GridBox(children=items, layout=widgets.Layout(grid_template_columns='repeat(4, 180px)'))
        display(grid)

def update_ref_options(change=None):
    names = []
    for i, f in enumerate(files_w.value):
        # 파일명 추출 개선
        if hasattr(f, 'name') and f.name:
            name = f.name
        else:
            meta = f.get('metadata') or {}
            name = meta.get('name', f'image_{i+1}.png')
        names.append(name)
    
    ref_dropdown.options = names
    if names: 
        ref_dropdown.value = names[0]
    render_thumbnails()

files_w.observe(update_ref_options, names='value')
ref_dropdown.observe(render_thumbnails, names='value')
 # 평가 방식 설명 추가
info_panel = widgets.HTML(
            value="""
            <div style='background-color: #f8f9fa; padding: 15px; border-radius: 8px; border: 1px solid #dee2e6; margin-top: 10px;'>
                <h4 style='margin-top: 0; color: #495057;'>평가 방식 설명</h4>
                <ul style='margin-bottom: 0; color: #6c757d; line-height: 1.6;'>
                    <li><strong>CLIP 점수:</strong> 텍스트-이미지 유사도 (0.4 이상 → 1.0, 아니면 (score+1)/1.4)</li>
                    <li><strong>BLIP2 점수:</strong> 토픽 유사도 (0.7 이상 → 1.0, 아니면 score/0.7)</li>
                    <li><strong>LPIPS 점수:</strong> 이미지 시각적 유사도 (낮을수록 좋음, 1-score로 정규화)</li>
                    <li><strong>Final Score:</strong> 가중평균 (CLIP:20%, BLIP2:50%, LPIPS:30%) × 100</li>
                    <li><strong>참고:</strong> 기준 이미지는 LPIPS 제외하고 계산됩니다</li>
                </ul>
            </div>
        """,
        layout=widgets.Layout(width='800px')
    )
display(
    topic_w, 
    prompt_w, 
    files_w, 
    ref_dropdown, 
    widgets.HBox([enable_clip_w, enable_blip2_w, enable_lpips_w]), 
    thumbs_out, 
    run_btn, 
    info_panel,  # 정보 패널 추가
    out
)


Textarea(value='A wooden cabin with a smoking chimney stands among snow-covered trees and a snowy mountain', d…

Textarea(value='Only one wooden cabin with one chimney, no other buildings or cabins, in a snowy forest with m…

FileUpload(value=({'name': '1.png', 'type': 'image/png', 'size': 1845726, 'content': <memory at 0x7f55b8c57a00…

Dropdown(description='Reference Image', index=1, options=('1.png', '2.png', '3.png', '4.png'), style=Descripti…

Output()

Button(button_style='primary', description='Run Evaluation', style=ButtonStyle())

HTML(value="\n            <div style='background-color: #f8f9fa; padding: 15px; border-radius: 8px; border: 1p…

Output()